In [251]:
from __future__ import print_function
import pandas as pd
import numpy as np
import math
#Vamos importar o solver linear do ORTOOLS

from ortools.linear_solver import pywraplp
#Vamos importar o numpy como uma ferramenta auxiliar 


solver = pywraplp.Solver.CreateSolver('GLOP')

def mi2ai(i,j,line_length):
	return i*N+j

def ai2mi(index, line_length):
	return [maph.floor(index/line_length), index%line_length]
sla = {}
'''OMA = list()
NOMA = list()
FixNOMA = list()
OUTAGE = list()
NOMA_PAIRS = list()
FNOMA_PAIRS = list()
'''

'OMA = list()\nNOMA = list()\nFixNOMA = list()\nOUTAGE = list()\nNOMA_PAIRS = list()\nFNOMA_PAIRS = list()\n'

In [252]:
NUFNOMA = pd.read_csv('0DF4.csv')
NUFNOMA = NUFNOMA.drop(['Unnamed: 0'],axis=1)
NUFNOMA.index = NUFNOMA.columns
asd = sorted(list(NUFNOMA.columns))
NUFNOMA = NUFNOMA.reindex(asd,axis=1)
NUFNOMA = NUFNOMA.reindex(asd,axis=0)
print(NUFNOMA.head(10))

           0         1          2
0  32.000000  0.000000   0.000000
1   2.798949  4.000000  14.421977
2   2.798949  7.298864   1.000000


In [253]:
FNOMA = pd.read_csv('0DF3.csv')
FNOMA = FNOMA.drop(['Unnamed: 0'],axis=1)
FNOMA.index = FNOMA.columns
asd = sorted(list(FNOMA.columns))
FNOMA = FNOMA.reindex(asd,axis=1)
FNOMA = FNOMA.reindex(asd,axis=0)
FNOMA.head(10)

,0,1,2
0,32.000000,0.000000,0.000000
1,1.993112,4.000000,1.993112
2,1.999951,1.999951,1.000000


In [254]:
DATA = pd.read_csv('DATA.csv')
DATA

DF_OMA = pd.read_csv('0DF1.csv')

DF_OMA = DF_OMA.drop(['Unnamed: 0'],axis=1)
DF_OMA.index = DF_OMA.columns
asd = sorted(list(DF_OMA.columns))
DF_OMA = DF_OMA.reindex(asd,axis=1)
DF_OMA = DF_OMA.reindex(asd,axis=0)
#print(DF_OMA)

DF_NOMA = pd.read_csv('0DF.csv')
DF_NOMA = DF_NOMA.drop(['Unnamed: 0'],axis=1)
DF_NOMA.index = DF_NOMA.columns
asd = sorted(list(DF_NOMA.columns))
DF_NOMA = DF_NOMA.reindex(asd,axis=1)
DF_NOMA = DF_NOMA.reindex(asd,axis=0)
#DF_NOMA.head(10)

DF2 = pd.read_csv('0DF2.csv')
DF2['Unnamed: 0'] = DF2.columns[1:]
DF2.rename(columns = {'Unnamed: 0':'SUMRATE FU-NU'}, inplace = True)
DF2 = DF2.set_index(DF2['SUMRATE FU-NU'])
DF2 = DF2.drop(['SUMRATE FU-NU'],axis=1)
asd = sorted(list(DF2.columns))
DF2 = DF2.reindex(asd,axis=1)
DF2 = DF2.reindex(asd,axis=0)
#print(DF2.head(10))




sub_util = {}
sumrate = {}
i=0
for c in DF2.columns:
    #print(DF2[c][i])
    #print(np.diag(DF_OMA)[i])
    if DF2[c][i] < np.diag(DF_OMA)[i]:
        sub_util[c] = DF2[c]/np.diag(DF_OMA)[i]
        sumrate[c] = DF_NOMA[c]
    else:
        #print(f'{c}alo')
        sub_util[c] = 0
        sumrate[c] = 0
    #desp[c] = DF[c]/np.diag(DF2)[i]
    i=i+1

sub_util = pd.DataFrame(sub_util)
sumrate = pd.DataFrame(sumrate) 
sumrate

FU = []
N = len(sub_util.columns)
Aloc = sumrate
for i in range(N):
    for j in range(N):
        
        if sumrate.iloc[i][j] > 0.0:
            Aloc.iloc[i,j] = DF2.iloc[i][j]
        
        if sumrate.iloc[i][i] == 0:
             Aloc.iloc[i,j] = DF_OMA.iloc[i,j]


sub_util = {}
sumrate = {}
i=0
for c in Aloc.columns:
    #print(DF2[c][i])
    #print(np.diag(DF_OMA)[i])
    if DF2[c][i] < np.diag(DF_OMA)[i]:
        sub_util[c] = Aloc[c]/np.diag(DF_OMA)[i]
        sumrate[c] = DF_NOMA[c]
    else:
        #print(f'{c}alo')
        sub_util[c] = 0
        sumrate[c] = 0
    #desp[c] = DF[c]/np.diag(DF2)[i]
    i=i+1


sub_util = pd.DataFrame(sub_util)
sumrate = pd.DataFrame(sumrate) 
#sumrate    '''

In [255]:
sumrate

,0,1,2
0,0,0.000000,0.000000
1,0,4.000000,9.281729
2,0,5.382113,1.000000


In [256]:
FU = []
N = len(sub_util.columns)
Aloc = sumrate
for i in range(N):
    for j in range(N):
        
        if sumrate.iloc[i][j] > 0.0:
            Aloc.iloc[i,j] = DF2.iloc[i][j]
        
        if sumrate.iloc[i][i] == 0:
             Aloc.iloc[i,j] = DF_OMA.iloc[i,j]
Aloc             

,0,1,2
0,4.634439,0.000000,0.000000
1,0.000000,4.000000,10.281729
2,0.000000,9.382113,1.000000


Criando as Variaveis: N Variaveis de usuários primários
                      NxN Variaveis de usuários secundários

In [257]:
Aloc1 =Aloc #pd.read_csv('SUMRATE_DB.csv',index_col=0)
NU = []
FU = []
N = len(sub_util.columns)
# Matriz de interesses
A = Aloc.values   
#A = Aloc1.values
# i*j variaveis serao necessarias
'''
for j in range(0,N):
		NU.append(solver.NumVar(0, 1, f'NU_{j}'))
print ('Numero de variaveis =', solver.NumVariables())
'''
for i in range(0,N):
	for j in range(0,N):
		FU.append(solver.NumVar(0, 1,f'FU_{i}_{j}'))
#print ('Numero de variaveis =', solver.NumVariables())



In [258]:
A

array([[ 4.63443853,  0.        ,  0.        ],
       [ 0.        ,  4.        , 10.28172877],
       [ 0.        ,  9.38211323,  1.        ]])

# Restrições: 

1ª restrição: O número de usuários primários não pode ser superior ao número usuários total

2ª restrição: O número de usuários secundários não pode ser superior ao número usuários total

3ª restrição: Um usuário primário não pode ser associado como um usuário secundário.

4ª restrição: Um usuário secundário não pode ser associado como um usuário primário.

In [259]:
head = 1


for j in range(N):
    R = '0 <= '
    ct = solver.Constraint(1,2, str(head))

    for i in range(N):
        if i == j:
            ct.SetCoefficient(FU[mi2ai(j,j,N)], 1)
            R += f'{FU[mi2ai(j,j,N)]} + '
        else:    
            ct.SetCoefficient(FU[mi2ai(i,j,N)], 1)
            R += f'{FU[mi2ai(i,j,N)]} + '
            ct.SetCoefficient(FU[mi2ai(j,i,N)], 1)
            R += f'{FU[mi2ai(j,i,N)]} + '
    head+=1
    #print(R)    

ct = solver.Constraint(N,N, str(head)) 
for j in range(N):
    R = '0 <= '
    
    ct.SetCoefficient(FU[mi2ai(j,j,N)], 1)  

 
for i in range(N):
    R = '0 <= '
    ct = solver.Constraint(0,2, str(head)) 
    for j in range(N):
        ct.SetCoefficient(FU[mi2ai(i,j,N)], 1)     
    head +=1
solver.NumConstraints()

7

objective = solver.Objective()
EQ = 'EQ = '
for j in range(N):
    objective.SetCoefficient(NU[j], np.diag(DF_NOMA)[j])
    EQ += f'{NU[j]}*{np.diag(DF_NOMA)[j]} + '
    for i in range(N):
        if i == j:
            objective.SetCoefficient(FU[mi2ai(i,j,N)], int(DF_NOMA.iloc[i,j]))
            EQ += f'{FU[mi2ai(i,j,N)]}*{int(DF_NOMA.iloc[i,j])} + ' 
        else:
            objective.SetCoefficient(FU[mi2ai(i,j,N)], int(DF_NOMA.iloc[i,j]))
            EQ += f'{FU[mi2ai(i,j,N)]}*{int(DF_NOMA.iloc[i,j])} + '     
#print(EQ)
objective.SetMaximization()
solver.Solve()

In [260]:
objective = solver.Objective()
for j in range(N):
    #objective.SetCoefficient(FU[mi2ai(i,i,N)], 1)
    for i in range(N):
        objective.SetCoefficient(FU[mi2ai(i,j,N)], int(Aloc1.iloc[i,j]))
        #ct.SetCoefficient(FU[mi2ai(j,i,N)], 1)
objective.SetMaximization()

# #Executaremos o solver
solver.Solve()

0

In [261]:
#FU[mi2ai(0,1,N)]

In [262]:
#DF_NOMA = pd.read_csv('NOMA_DB.csv',index_col=0)
#DF_NOMA

In [263]:
A

array([[ 4.63443853,  0.        ,  0.        ],
       [ 0.        ,  4.        , 10.28172877],
       [ 0.        ,  9.38211323,  1.        ]])

In [264]:
print('Solution:')
#print('Valor objetivo =', (objective.Value()))
pair = 0
matriz = np.zeros(shape=(N,N))
for i in range(0,N):
    for j in range(0,N):
        matriz[i,j] = (int(FU[mi2ai(i,j,N)].solution_value()))
for i in range(0,N):
    for j in range(0,N):
        if i != j:
            matriz[i,j] = (int(FU[mi2ai(i,j,N)].solution_value()))
            if int(FU[mi2ai(i,j,N)].solution_value()) ==1:
                    matriz[i,i] = 0
                    matriz[j,j] = 0
usuario_atendido = list(DF_NOMA.columns)
tot = 0
ftot = 0
outage = 0
NomaP= 0
FNomaP = 0
for i in range(0,N):
	for j in range(0,N):
		if matriz[i,j] ==1:
			if i == j:
				print(f'OMA User: {DF_NOMA.columns[j]}, data rate = {Aloc.iloc[i,i]}')
				print(f'Used {(Aloc.iloc[i,j]/np.diag(Aloc)[i])*100}% from Shannon Channel Capacity')
				usuario_atendido.remove(DF_NOMA.columns[j])
				tot +=Aloc1.iloc[i,j]
				ftot += Aloc1.iloc[i,i]
				#ftot += FNOMA.iloc[i,j]
			else:
				print(f'NOMA Pair {pair} Near  User: {FU[j]}, data rate = {Aloc.iloc[j,j]}') 
				print(f'NOMA Pair {pair} Far  User: {FU[i]}, data rate = {DF_NOMA.iloc[i,j]}')
				NomaP +=1
				print(f'FIXED NOMA Pair {pair} Near  User: {FU[j]}, data rate = {NUFNOMA.iloc[i,j]}') 
				print(f'FIXED NOMA Pair {pair} Far  User: {FU[i]}, data rate = {FNOMA.iloc[i,j]}')
				#print(f'Gain over OMA:{(Aloc.iloc[i,j]/np.diag(Aloc)[j])*100}% ') 
				tot +=Aloc1.iloc[j,j]
				if FNOMA.iloc[i,j] >= Aloc1.iloc[i,i] and NUFNOMA.iloc[j,j] >= Aloc1.iloc[j,j]:
					print(f'alocado:FIXED NOMA Pair {pair} Near  User: {FU[j]}, data rate = {NUFNOMA.iloc[i,j]}') 
					print(f'alocado:FIXED NOMA Pair {pair} Far  User: {FU[i]}, data rate = {FNOMA.iloc[i,j]}')
					FNomaP +=1
					ftot += NUFNOMA.iloc[i,j]
					ftot += FNOMA.iloc[j,j]
					usuario_atendido.remove(DF_NOMA.columns[j])
					usuario_atendido.remove(DF_NOMA.index[i])
					 
				else:
					print(f'Não alocado:FIXED NOMA Pair {pair} Near  User: {FU[j]}, data rate = {NUFNOMA.iloc[i,j]}, Alocado Ortogonal: {Aloc1.iloc[j,j]}') 
					print(f'Não alocado:FIXED NOMA Pair {pair} Far  User: {FU[i]}, data rate = {FNOMA.iloc[i,j]}, Alocado Ortogonal: {Aloc1.iloc[i,i]}')
					outage +=1
					print(f'OMA User: {DF_NOMA.columns[j]}, data rate = {Aloc.iloc[i,i]}')
					print(f'Used {(Aloc.iloc[i,j]/np.diag(Aloc)[i])*100}% from Shannon Channel Capacity')
					#usuario_atendido.remove(DF_NOMA.columns[j])
					tot +=Aloc1.iloc[i,j]
					ftot += Aloc1.iloc[i,i]
					ftot += Aloc1.iloc[j,j]
					pass
				pair = pair +1
		else:
			pass
print(matriz)
print(f'Fixed PA NOMA SumRate = {ftot}')
print('Total SumRate =', np.sum(np.sum(Aloc1*matriz)))
print(f'OMA Total SumRate = {np.sum(np.sum(np.diag(Aloc1)))}')


Solution:
OMA User: 0, data rate = 4.634438530519827
Used 100.0% from Shannon Channel Capacity
NOMA Pair 0 Near  User: FU_0_2, data rate = 1.0
NOMA Pair 0 Far  User: FU_0_1, data rate = 9.281728765640628
FIXED NOMA Pair 0 Near  User: FU_0_2, data rate = 14.421977469018186
FIXED NOMA Pair 0 Far  User: FU_0_1, data rate = 1.993111969103155
Não alocado:FIXED NOMA Pair 0 Near  User: FU_0_2, data rate = 14.421977469018186, Alocado Ortogonal: 1.0
Não alocado:FIXED NOMA Pair 0 Far  User: FU_0_1, data rate = 1.993111969103155, Alocado Ortogonal: 4.0
OMA User: 2, data rate = 4.0
Used 257.0432191410157% from Shannon Channel Capacity
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
Fixed PA NOMA SumRate = 9.634438530519827
Total SumRate = 14.916167296160454
OMA Total SumRate = 9.634438530519827


In [265]:
Aloc

,0,1,2
0,4.634439,0.000000,0.000000
1,0.000000,4.000000,10.281729
2,0.000000,9.382113,1.000000


In [266]:
OMA.append(np.sum(np.sum(np.diag(Aloc))))
NOMA.append(np.sum(np.sum(Aloc*matriz)))
FixNOMA.append(ftot)
NOMA_PAIRS.append(NomaP)
FNOMA_PAIRS.append(FNomaP)
OUTAGE.append(outage)

NameError: name 'OMA' is not defined

In [ ]:
sla['OMA Sum rate'] = OMA
sla['Architecture Sum rate'] = NOMA
sla['Fixed NOMA Sum rate'] = FixNOMA
sla['Fixed NOMA Pairs'] = FNOMA_PAIRS
sla['NOMA Pairs'] = NOMA_PAIRS
sla['Fixed NOMA Outage'] = OUTAGE

In [ ]:
#Aloc #sumrate

In [ ]:
#DF_NOMA #achievable data rate

In [ ]:
#FNOMA
#DF_OMA #max channel capacity

In [ ]:
len(sla['OMA Sum rate'])

19

In [ ]:
sla['OMA Sum rate'][-1]

89.40946162205923

In [ ]:
sla['Architecture Sum rate'][-1]

149.37326967758014

In [ ]:
sla['Fixed NOMA Sum rate'][-1]

118.85209121514924

In [ ]:
KCT = pd.DataFrame(sla)
KCT.to_csv('Arq.csv',)

In [ ]:
#OUTAGE